# Run alternative flux modes simulations

In [ ]:
# | default_exp runalternativemodes

As with runconstraintscan.py, this notebook can either be run as a notebook, or using nbdev a python script is produced that can be run in the command line for easier running on e.g. a cluster. It is a notebook in the nbs directory that is symlinked in the nbs/functions directory in order to be exported as a .py file in the mmon-gcm library.

NOTE: I have had issues where for an unknown reason the code hangs while solving. In this case I can check the temporary files to see which solutions are missing and run it again to just get those. For the paper, the blue light, unconstrained, KO solution number 348 hangs with a tolerance of 1e-8 and says infeasible for a tolerance of 1e-7. So the solution list is constructed from combining the remaining temporary files rather than all in one go. 

In [ ]:
# | export

import shutil
import sys
from pathlib import Path

import cobra
# from x import y syntax doesn't work because of nbdev export format
import mmon_gcm.alternativemodes
import mmon_gcm.buildingediting
import mmon_gcm.supermodel
import pandas as pd
from pandarallel import pandarallel

In [ ]:
# This cell isn't exported to the .py file, so define here if running in notebook rather than as .py on e.g.a cluster

sys.argv = ["script_name",
            "../outputs/alternative_weighting/test_solution.csv",
            "../models/4_stage_GC.json",
            "../outputs/alternative_weighting/alternative_weights.csv",
            "../inputs/arabidopsis_parameters.csv",
            "blue",
            "False",
            "True",
            "2"]

In [ ]:
# | export

results_path = sys.argv[1]
model_path = sys.argv[2]
weightings_csv = sys.argv[3]
parameters_csv = sys.argv[4]
light_colour = sys.argv[5]
atpase_constraint = sys.argv[6]
starch_knockout = sys.argv[7]
no_cores = int(sys.argv[8])

In [ ]:
# | export

if light_colour != "blue" and light_colour != "white" and light_colour != "nops":
    raise ValueError(f"Please specify either 'blue' or 'white' or 'nops' for light, not {light_colour}")

if atpase_constraint == "True":
    atpase_constraint = True
elif atpase_constraint == "False":
    atpase_constraint = False
else:
    raise ValueError(f"Please specify True or False for the ATPase constraint, not {atpase_constraint}")

if starch_knockout == "True":
    starch_knockout = True
elif starch_knockout == "False":
    starch_knockout = False
else:
    raise ValueError(f"Please specify True or False for the starck knockout, not {starch_knockout}")

In [ ]:
# | export

model = cobra.io.load_json_model(model_path)
print("Model imported")

Model imported


In [ ]:
# | export

print(model.solver.configuration.tolerances.integrality)
print(model.solver.configuration.tolerances.feasibility)
model.solver.configuration.tolerances.feasibility = 1e-8  # 1e-9 takes a long time to solve
print(model.solver.configuration.tolerances.feasibility)

1e-07
1e-07
1e-08


In [ ]:
# | export

parameters_df = pd.read_csv(parameters_csv, index_col=0)

In [ ]:
parameters_df

,Value,Units,Source
P_abs,9.000000e-01,Dimensionless,"Zhu, Long, and Ort (2010)"
T_l,1.700000e-04,m,Wuyts et al. (2010)
A_l,1.000000e+00,m$^2$,Fixed
V_gc_ind,4.750000e-13,dm$^3$,Jezek and Blatt (2017)
FqFm,9.000000e-01,Dimensionless,Lawson (2003)
R_ch,6.923077e-02,Dimensionless,"Fujiwara, Sanjaya, and Itoh (2019)"
L_air,3.700000e-01,Dimensionless,Earles et al. (2018)
L_epidermis,1.500000e-01,Dimensionless,Wuyts et al. (2010)
Vac_frac,7.510000e-01,Dimensionless,Wang et al. (2017)
T,2.961500e+02,K,Horrer et al. (2016)


In [ ]:
# | export

arabidopsis_supermodel = mmon_gcm.supermodel.SuperModel(parameters_df.loc[:, "Value"], fba_model=model)
arabidopsis_supermodel.constrain_osmolarity(printouts=False)
arabidopsis_supermodel.constrain_photons(150, printouts=False)
arabidopsis_supermodel.add_maintenance();

## Constrain light, ATPase, starch

In [ ]:
# | export

if light_colour == "blue":
    arabidopsis_supermodel.fba_model.reactions.Photon_tx_gc_2.upper_bound = 0
    arabidopsis_supermodel.fba_model.reactions.Photon_tx_me_2.upper_bound = 0
    print("Model constrained with blue light")
elif light_colour == "nops":
    mmon_gcm.buildingediting.set_bounds_multi(arabidopsis_supermodel.fba_model, "Photon_tx_gc", 0, 0)
    print("Photosynthesis prevented in guard cell")
else:
    print("Model constrained with white light")

if atpase_constraint == True:
    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.48)
    mmon_gcm.buildingediting.set_bounds_multi(arabidopsis_supermodel.fba_model,
                                              "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)
    print("Model ATPase constrained")
else:
    print("Model ATPase left unconstrained")

if starch_knockout == True:
    mmon_gcm.buildingediting.set_bounds_multi(arabidopsis_supermodel.fba_model, "RXN_1827_p_gc", 0, 0)
    print("Model starch knocked out")
else:
    print("Model starch left unconstrained")

print("Supermodel established and model constrained")

Model constrained with blue light
Model ATPase left unconstrained
Model starch knocked out
Supermodel established and model constrained


In [ ]:
# | export
weightings = pd.read_csv(weightings_csv, index_col=[0], header=[0])
print("Weightings file imported")

Weightings file imported


**If you are running from notebook you'll want to comment out the next cell! It's here so that running nbdev tests doesn't take too long, as they test running the entire notebook. It won't be exported so running as a script will be fine**

In [ ]:
weightings = weightings.iloc[0:4]  # so tests don't take too long
weightings.head()

,10_FORMYL_THF_pc_gc_1,10_FORMYL_THF_pc_gc_2,10_FORMYL_THF_pc_gc_3,10_FORMYL_THF_pc_gc_4,10_FORMYL_THF_pc_me_1,10_FORMYL_THF_pc_me_2,10_FORMYL_THF_pc_me_3,10_FORMYL_THF_pc_me_4,1TRANSKETO_RXN_p_gc_1,1TRANSKETO_RXN_p_gc_2,...,sSUC_biomass_me_3,sSUC_biomass_me_4,unlProtHYPO_c_gc_1,unlProtHYPO_c_gc_2,unlProtHYPO_c_gc_3,unlProtHYPO_c_gc_4,unlProtHYPO_c_me_1,unlProtHYPO_c_me_2,unlProtHYPO_c_me_3,unlProtHYPO_c_me_4
0,3.196322,0.266360,6.126284,3.196322,2.003810,0.166984,3.840636,2.003810,4.628290,0.385691,...,3.685919,1.923088,3.013075,0.251090,5.775060,3.013075,4.919640,0.409970,9.429310,4.919640
1,2.976393,0.248033,5.704753,2.976393,0.059372,0.004948,0.113797,0.059372,3.306903,0.275575,...,11.088275,5.785187,4.092069,0.341006,7.843133,4.092069,0.268098,0.022342,0.513855,0.268098
2,3.812475,0.317706,7.307244,3.812475,3.160252,0.263354,6.057151,3.160252,1.036738,0.086395,...,5.557520,2.899576,3.424889,0.285407,6.564371,3.424889,1.629429,0.135786,3.123073,1.629429
3,0.186226,0.015519,0.356934,0.186226,1.913905,0.159492,3.668317,1.913905,4.470282,0.372524,...,2.648911,1.382040,4.475259,0.372938,8.577580,4.475259,0.418057,0.034838,0.801276,0.418057


In [ ]:
# | export

temp_results = results_path.split(".")[0]+"_tmp/"
print(f"Creating temp directory {temp_results} to store solutions as they come in")
Path(temp_results).mkdir(parents=True, exist_ok=True)

Creating temp directory _tmp/ to store solutions as they come in


In [ ]:
# | export

pandarallel.initialize(nb_workers=no_cores, progress_bar=False)
print(f"Solving model for {len(weightings.index)} alternative weightings")
weightings_solution = weightings.parallel_apply(
    mmon_gcm.alternativemodes.solve_model_with_weightings, args=([arabidopsis_supermodel.fba_model, temp_results]), axis=1)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Solving model for 4 alternative weightings
Solving solution 0
Solving solution 2
Solving solution 3
Solving solution 1


In [ ]:
# | export

weightings_solution.to_csv(results_path)
if len(weightings_solution) == len(weightings):
    print(f"All solutions saved to {results_path}")
    print(f"Deleting temp directory {temp_results}")
    shutil.rmtree(temp_results)

All solutions saved to ../outputs/alternative_weighting/test_solution.csv
Deleting temp directory _tmp/
